In [2]:
import torch
from torch import nn,optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
from utils import Logger
import matplotlib.pyplot as plt

In [3]:
def mnist_data():
    compose = transforms.Compose(
        [transforms.ToTensor(),
        #  transforms.Normalize((.5, .5, .5), (.5, .5, .5))
        transforms.Normalize([0.5],[0.5])
        ])
    out_dir = './dataset'
    return datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)
# Load data
data = mnist_data()
# Create loader with data, so that we can iterate over it
data_loader = torch.utils.data.DataLoader(data, batch_size=100, shuffle=True)
# Num batches
num_batches = len(data_loader)

In [ ]:
def ss_data():
    return torch.load('../dataloader_32.pt')
data_loader = ss_data()
num_batches = len(data_loader)

In [4]:
class DiscriminatorNet(torch.nn.Module):
    """
    A three hidden-layer discriminative neural network
    """
    def __init__(self):
        super(DiscriminatorNet, self).__init__()
        n_features = 784
        n_out = 1
        
        self.hidden0 = nn.Sequential( 
            nn.Linear(n_features, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.out = nn.Sequential(
            torch.nn.Linear(256, n_out),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x
discriminator = DiscriminatorNet()

In [5]:
def images_to_vectors(images):
    return images.view(images.size(0), 784)

def vectors_to_images(vectors):
    return vectors.view(vectors.size(0), 1, 22, 28)

In [ ]:
def images_to_vectors(images):
    return images.view(images.size(0),44)

def vectors_to_images(vectors):
    return vectors.view(vectors.size(0), 1, 11, 4)

In [6]:
class GeneratorNet(torch.nn.Module):
    """
    A three hidden-layer generative neural network
    """
    def __init__(self):
        super(GeneratorNet, self).__init__()
        n_features = 100
        n_out = 784
        
        self.hidden0 = nn.Sequential(
            nn.Linear(n_features, 256),
            nn.LeakyReLU(0.2)
        )
        self.hidden1 = nn.Sequential(            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2)
        )
        
        self.out = nn.Sequential(
            nn.Linear(1024, n_out),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x
generator = GeneratorNet()

In [7]:
def noise(size):
    '''
    Generates a 1-d vector of gaussian sampled random values
    '''
    n = Variable(torch.randn(size, 100,dtype=torch.float32))
    return n


In [8]:
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)

In [9]:
loss = nn.BCELoss()

In [10]:
def ones_target(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1,dtype=torch.float32))
    return data

def zeros_target(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1,dtype=torch.float32))
    return data

In [11]:
def train_discriminator(optimizer, real_data, fake_data):
    N = real_data.size(0)
    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on Real Data
    prediction_real = discriminator(real_data)
    # Calculate error and backpropagate
    error_real = loss(prediction_real, ones_target(N) )
    error_real.backward()

    # 1.2 Train on Fake Data
    prediction_fake = discriminator(fake_data)
    # Calculate error and backpropagate
    error_fake = loss(prediction_fake, zeros_target(N))
    error_fake.backward()
    
    # 1.3 Update weights with gradients
    optimizer.step()
    
    # Return error and predictions for real and fake inputs
    return error_real + error_fake, prediction_real, prediction_fake

In [12]:
def train_generator(optimizer, fake_data):
    N = fake_data.size(0)
    # Reset gradients
    optimizer.zero_grad()
    # Sample noise and generate fake data
    prediction = discriminator(fake_data)
    # Calculate error and backpropagate
    error = loss(prediction, ones_target(N))
    error.backward()
    # Update weights with gradients
    optimizer.step()
    # Return error
    return error

In [13]:
num_test_samples = 16
test_noise = noise(num_test_samples)

In [17]:
# Create logger instance
logger = Logger(model_name='VGAN', data_name='MNIST')
# Total number of epochs to train
num_epochs = 200
for epoch in range(num_epochs):
    for n_batch, (real_batch,_) in enumerate(data_loader):
        N = real_batch.size(0)
        # print('real')
        # print(real_batch.dtype)
        # 1. Train Discriminator
        real_data = Variable(images_to_vectors(real_batch))
        # Generate fake data and detach 
        # (so gradients are not calculated for generator)
        fake_data = generator(noise(N)).detach()
        # Train D
        d_error, d_pred_real, d_pred_fake = \
              train_discriminator(d_optimizer, real_data, fake_data)

        # 2. Train Generator
        # Generate fake data
        fake_data = generator(noise(N))
        # Train G
        g_error = train_generator(g_optimizer, fake_data)
        # Log batch error
        logger.log(d_error, g_error, epoch, n_batch, num_batches)
        # Display Progress every few batches
        if (n_batch) % 100 == 0: 
            test_images = vectors_to_images(generator(test_noise))
            test_images = test_images.data
            logger.log_images(
                test_images, num_test_samples, 
                epoch, n_batch, num_batches
            );
            # Display status Logs
            logger.display_status(
                epoch, num_epochs, n_batch, num_batches,
                d_error, g_error, d_pred_real, d_pred_fake
            )

RuntimeError: shape '[16, 1, 22, 28]' is invalid for input of size 12544

In [16]:
for i,(batch,_) in enumerate(data_loader):
    if i == 3:
        break
    plt.figure()
    plt.imshow(batch[0])
    plt.colorbar()

TypeError: Invalid shape (1, 28, 28) for image data

In [ ]:
test_data = generator(noise(1))

In [ ]:
test_data = generator(noise(10))
plt.imshow(test_data[6].reshape(11,4).detach().numpy())
plt.colorbar()

In [ ]:
generator(noise(1)).mean()

In [ ]:
noise(1)

In [ ]:
generator(noise(1))

In [ ]:
tt1 = noise(1)
print(tt1)
t1 = generator(tt1)
print(t1)
print(t1.mean())
print(t1.std())

In [ ]:
tt2 = noise(1)
print(tt2)
t2 = generator(tt2)
print(t2)
print(t2.mean())
print(t2.std())

In [ ]:
t2-t2.mean()

In [ ]:
discriminator(test_data[0])

In [ ]:
discriminator(test_data)

In [ ]:
discriminator(real_batch[0]-real_batch[0].mean())

In [ ]:
plt.imshow((real_batch[0]-real_batch[0].mean()).reshape(11,4))
plt.colorbar()

In [ ]:
means = []
for i in range(len(real_batch)):
    means.append(real_batch[i].mean())

In [ ]:
import numpy as np

In [ ]:
means[3]

In [ ]:
discriminator(real_batch[0])

In [ ]:
real_batch[0].mean()

In [ ]:
real_batch[0].std()

In [ ]:
means = real_batch.mean(axis=1)

In [ ]:
ff = (real_batch.T-means).T

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
tmp1 = real_batch[0] - real_batch[0].mean()
print(tmp1)
print(real_batch[0].std())

In [ ]:
tmp2 = real_batch[1] - real_batch[1].mean()
print(tmp2)
print(real_batch[1].mean())
print(real_batch[1])
print(real_batch[1].std())

In [ ]:
real_batch.std(axis=1)
plt.hist(real_batch.std(axis=1),bins=100)

In [ ]:
real_batch.std(axis=1)
plt.hist(real_batch.mean(axis=1),bins=100)